# LangChain Agent and SQL query:

In this notebook, we see how to create a LangChain agent to execute SQL queries on an SQLite database using Python. This guide covers setting up an SQLite database with SQLAlchemy, populating it with sample data, and creating a LangChain agent to query the database.

### Environment Setup:

Install necessary packages (langchain, sqlalchemy, ...)

In [ ]:
!pip install --upgrade --quiet langchain-community langgraph langchain-anthropic sqlalchemy sqlite tavily-python langchain-google-genai google-search-results pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement sqlite (from versions: none)
ERROR: No matching distribution found for sqlite


Setting an API keys:

In [ ]:
from google.colab import userdata
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

###Initialize the Model:



In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage


model = ChatGoogleGenerativeAI(model="gemini-pro")

messages = [
    HumanMessage(content="The capital of Irland"),
]

response = model.invoke(messages)
response

AIMessage(content='Dublin', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0573dcbf-75dd-41ae-9099-17aef9994eb4-0', usage_metadata={'input_tokens': 5, 'output_tokens': 1, 'total_tokens': 6})

Note that the response from the model is an AIMessage, which includes a string response along with additional metadata. Often, we might only need the string response. To extract just the string, we can use a simple output parser.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

parser.invoke(response)

'Dublin'

In [ ]:
chain = model | parser
chain.invoke("The capital of Irland")

'Dublin'

###Define Tools:

LangChain offers a variety of tools that extend the functionality of language models, allowing them to interact with external APIs. Here we use Tavily and SerpAPI tools.



In [154]:
# ====== Tavily =======
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)

[{'url': 'https://world-weather.info/forecast/usa/san_francisco/june-2024/', 'content': 'Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ San Francisco Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info.'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1719166882, 'localtime': '2024-06-23 11:21'}, 'current': {'last_updated_epoch': 1719166500, 'last_updated': '2024-06-23 11:15', 'temp_c': 19.4, 'temp_f': 66.9, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 300, 'wind_dir': 'WNW', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.01, 'precip_in': 0.0, 'humidity': 66, 'cloud':

In [162]:
# ====== Serp API =======
from langchain_community.utilities import SerpAPIWrapper

search = SerpAPIWrapper()
search.run("weather in SF")

"{'type': 'weather_result', 'temperature': '64', 'unit': 'Fahrenheit', 'precipitation': '2%', 'humidity': '68%', 'wind': '11 mph', 'location': 'San Francisco, CA', 'date': 'Sunday 11:00 AM', 'weather': 'Partly cloudy'}"

### Using tools in Agents

In [165]:
# ====== Tavily =======
# from langchain_community.tools.tavily_search import TavilySearchResults

# search = TavilySearchResults(max_results=2)
# tools = [search]

# ====== Serp API =======
from langchain.agents import load_tools

tools = load_tools(["serpapi"])

In [166]:
model_with_tools = model.bind_tools(tools)

In [167]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hi there! How can I help you today?
ToolCalls: []


In [168]:
response = model_with_tools.invoke([HumanMessage(content="When was the last match of maple leafs?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'Search', 'args': {'__arg1': 'When was the last match of maple leafs?'}, 'id': 'a085d179-c641-4e29-a5a1-32ee2f5a6174'}]


In [169]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [170]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='d6d04094-f879-435a-8f6e-00b6ca6dc7bc'),
 AIMessage(content='Hello! How can I help you today?', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-3c494372-0dba-4533-9fc4-c04d60f5e4b7-0', usage_metadata={'input_tokens': 60, 'output_tokens': 9, 'total_tokens': 69})]

In [171]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="weather in SF")]}
)
response["messages"]

[HumanMessage(content='weather in SF', id='991c58b9-46a4-45ed-8076-18193af7346c'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'Search', 'arguments': '{"__arg1": "weather in SF"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-c026a2d4-9462-442a-a54d-734862362f34-0', tool_calls=[{'name': 'Search', 'args': {'__arg1': 'weather in SF'}, 'id': '85d77a67-d073-4824-8dbc-a68c8777f91e'}], usage_metadata={'input_tokens': 61, 'output_tokens': 16, 'total_tokens': 77}),
 ToolMessage(content="{'type': 'weather_result', 't

In [172]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'Search', 'arguments': '{"__arg1": "weather in sf"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0c96f9e3-c743-4f7a-ba93-0cf1bc3a7149-0', tool_calls=[{'name': 'Search', 'args': {'__arg1': 'weather in sf'}, 'id': '001e215f-2891-40cb-be64-10bb1e43c52f'}], usage_metadata={'input_tokens': 62, 'output_tokens': 16, 'total_tokens': 78})]}}
----
{'tools': {'messages': [ToolMessage(content="{'type': 'weather_result', 'temperature': '60', 'unit': 'Fa

In [173]:
response["messages"][-1].content

'The weather in SF is partly cloudy with a temperature of 64 degrees Fahrenheit. There is a 2% chance of precipitation, 68% humidity, and 11 mph wind.'

### Adding Memory

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Hi, My name is Majid!")]}, config)
response['messages'][-1].content

"It's nice to meet you, Majid. Is there anything I can help you with today?"

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="what's my name?")]}, config)
response['messages'][-1].content

'You are Majid.'

###Create DataBase

In [174]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [175]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)

In [176]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [177]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [178]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [179]:
for obs in observations:
    insert_obs(obs)

In [181]:
from sqlalchemy import text
with engine.connect() as connection:
    # Define the raw SQL query
    sql_query = text("SELECT * FROM stocks WHERE price > :price_threshold")

    # Execute the query with a parameter
    result = connection.execute(sql_query, {"price_threshold": 300})

    for row in result:
        print(row)

(6, 'XYZ', 810.0, '2023-01-01')
(7, 'XYZ', 803.0, '2023-01-02')
(8, 'XYZ', 798.0, '2023-01-03')
(9, 'XYZ', 795.0, '2023-01-04')
(10, 'XYZ', 791.0, '2023-01-05')


In [183]:
from langchain.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

db = SQLDatabase(engine)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=8
)

In [184]:
agent_executor.invoke("list of entries with price more than 300")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: stocksAction: sql_db_query_checker
Action Input: SELECT * FROM stocks WHERE price > 300;SELECT * FROM stocks WHERE price > 300;Action: sql_db_query
Action Input: SELECT * FROM stocks WHERE price > 300;[(6, 'XYZ', 810.0, '2023-01-01'), (7, 'XYZ', 803.0, '2023-01-02'), (8, 'XYZ', 798.0, '2023-01-03'), (9, 'XYZ', 795.0, '2023-01-04'), (10, 'XYZ', 791.0, '2023-01-05')]Final Answer: [(6, 'XYZ', 810.0, '2023-01-01'), (7, 'XYZ', 803.0, '2023-01-02'), (8, 'XYZ', 798.0, '2023-01-03'), (9, 'XYZ', 795.0, '2023-01-04'), (10, 'XYZ', 791.0, '2023-01-05')]

> Finished chain.


{'input': 'list of entries with price more than 300',
 'output': "[(6, 'XYZ', 810.0, '2023-01-01'), (7, 'XYZ', 803.0, '2023-01-02'), (8, 'XYZ', 798.0, '2023-01-03'), (9, 'XYZ', 795.0, '2023-01-04'), (10, 'XYZ', 791.0, '2023-01-05')]"}

In [186]:
agent_executor.invoke(
    "What is the multiplication of the price of ABC and XYZ stock tickers on Jan 1, 2023"
)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: stocksAction: sql_db_schema
Action Input: stocks
CREATE TABLE stocks (
	obs_id INTEGER NOT NULL, 
	stock_ticker VARCHAR(4) NOT NULL, 
	price FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from stocks table:
obs_id	stock_ticker	price	date
1	ABC	200.0	2023-01-01
2	ABC	208.0	2023-01-02
3	ABC	232.0	2023-01-03
*/Action: sql_db_query
Action Input: SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-01';[(200.0,)]Action: sql_db_query
Action Input: SELECT price FROM stocks WHERE stock_ticker = 'XYZ' AND date = '2023-01-01';[(810.0,)]I now know the final answer
Final Answer: 162000.0

> Finished chain.


{'input': 'What is the multiplication of the price of ABC and XYZ stock tickers on Jan 1, 2023',
 'output': '162000.0'}

### Adding Memory

In [205]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [206]:
from langchain.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

db = SQLDatabase(engine)
agent_executor = create_sql_agent(
    llm=llm,
    verbose=True,
    memory = memory,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=8,
)

In [208]:
agent_executor.invoke(
    "What was the price of ticker ABC on Jan 1, 2023?"
)



> Entering new SQL Agent Executor chain...
I can query the stock_prices table for the price of ticker ABC on Jan 1, 2023.
Action: sql_db_query
Action Input: SELECT price FROM stock_prices WHERE ticker = 'ABC' AND date = '2023-01-01'Error: (sqlite3.OperationalError) no such table: stock_prices
[SQL: SELECT price FROM stock_prices WHERE ticker = 'ABC' AND date = '2023-01-01']
(Background on this error at: https://sqlalche.me/e/20/e3q8)The stock_prices table does not exist, so I should check the schema of the database to find the correct table name.
Action: sql_db_list_tables
Action Input: stocksThe correct table name is stocks.
Action: sql_db_query
Action Input: SELECT price FROM stocks WHERE ticker = 'ABC' AND date = '2023-01-01'Error: (sqlite3.OperationalError) no such column: ticker
[SQL: SELECT price FROM stocks WHERE ticker = 'ABC' AND date = '2023-01-01']
(Background on this error at: https://sqlalche.me/e/20/e3q8)

The column name is incorrect, so I should check the schema of the table to find the correct column name.
Action: sql_db_schema
Action Input: stocks
CREATE TABLE stocks (
	obs_id INTEGER NOT NULL, 
	stock_ticker VARCHAR(4) NOT NULL, 
	price FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from stocks table:
obs_id	stock_ticker	price	date
1	ABC	200.0	2023-01-01
2	ABC	208.0	2023-01-02
3	ABC	232.0	2023-01-03
*/

The correct column name is stock_ticker.
Action: sql_db_query
Action Input: SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-01'[(200.0,)]

I now know the final answer
Final Answer: 200.0

> Finished chain.


{'input': 'What was the price of ticker ABC on Jan 1, 2023?',
 'output': '200.0'}

In [210]:
agent_executor.invoke(
    "What was XYZ stock price on that date?"
)



> Entering new SQL Agent Executor chain...
I need to know the correct table and column names to use.
Action: sql_db_list_tables
Action Input: stocksI should check the schema of the stocks table.
Action: sql_db_schema
Action Input: stocks
CREATE TABLE stocks (
	obs_id INTEGER NOT NULL, 
	stock_ticker VARCHAR(4) NOT NULL, 
	price FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from stocks table:
obs_id	stock_ticker	price	date
1	ABC	200.0	2023-01-01
2	ABC	208.0	2023-01-02
3	ABC	232.0	2023-01-03
*/I can now query the database for the stock price.
Action: sql_db_query
Action Input: SELECT price FROM stocks WHERE stock_ticker = 'XYZ' AND date = '2023-01-01';[(810.0,)]I now know the final answer
Final Answer: 810.0

> Finished chain.


{'input': 'What was XYZ stock price on that date?', 'output': '810.0'}